In [59]:
!poetry env use 3.12
!poetry env activate

Using virtualenv: C:\Users\04n\AppData\Local\pypoetry\Cache\virtualenvs\corn-leaf-disease-classifier-davY0eoL-py3.12
"C:\Users\04n\AppData\Local\pypoetry\Cache\virtualenvs\corn-leaf-disease-classifier-davY0eoL-py3.12\Scripts\activate.bat"


In [60]:
!poetry install
!poetry lock
#!poetry run pip freeze
    # ipykernel
    # keras
    # matplotlib
    # numpy
    # ruff
    # seaborn
    # tensorflow

Installing dependencies from lock file

No dependencies to install or update
Resolving dependencies...


# Setup Imports

In [61]:
import keras
import numpy as np # linear algebra
import tensorflow as tf

In [62]:
from numpy._typing._array_like import NDArray
from pathlib import Path
from PIL import Image, ImageEnhance
from typing import Any

IMAGE_SIZE: tuple[int, int] = (64, 64)
IMAGE_SUFFIXES: set[str] = {'.jpeg', '.jpg', '.png'}

# Load Datasets

In [63]:
ROOT_PATH: Path = Path.cwd().parent
TRAIN_PATH: Path = ROOT_PATH / 'data' / 'train'
TEST_PATH: Path = ROOT_PATH / 'data' / 'test'

In [64]:
def load_image_from_path(image_path: Path) -> list[int]:
    image: Image.Image = Image.open(image_path).convert('RGB').resize((64,64))
    image = ImageEnhance.Color(image).enhance(2.0)

    image_flat: NDArray[Any] = np.asarray(image, dtype="int64").flatten()

    ret: list[int] = [int(i) for i in image_flat]
    return ret

In [65]:
def get_dataset_classes(dataset_path: Path) -> list[str]:
    return [d.name for d in dataset_path.iterdir() if d.is_dir()]

# Load Model from File

In [66]:
model_path: Path = Path.cwd() / 'final' / 'model.keras'

In [67]:
model = keras.saving.load_model(model_path)

# Predict Image

In [68]:
def predict_image(image_path: Path) -> str:
    image_raw: list[int] = load_image_from_path(image_path)
    image: NDArray[Any] = np.array(image_raw).reshape(1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3)

    pred = model.predict(image)
    pred_class = np.argmax(pred, axis = 1) 
    prediction: list[str] = [get_dataset_classes(TRAIN_PATH)[i] for i in pred_class]

    return prediction[0]

In [69]:
predict_image(TRAIN_PATH / 'Blight' / 'b1.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


'Gray_Leaf_Spot'